<a href="https://colab.research.google.com/github/tniccum21/Cards/blob/master/simple_cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
"""
Simple cards - a platform for NN learning to play simple trick-based card games

Current status - platform for dealing hands and playing semi-random game built
To do:  build NN to learn/play

"""


import numpy as np
import random
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import time 

CARD_PLAYED = 20
DECK_SIZE = 20


"""
NN stuff
"""

#########################################################################
##  CLASS State
##    contains the complete state of the game as seen by one player
##    high-level:
##      records:
##        the trump suit
##        players current hand
##        recording of each previous round of play in the game, and the order played
##
##    initial constructor initializes the game state to zeros
##    print - prints a nice copy of the situation
##
##
class State:
    # State structure - applies to one game
    # [0]     - trump for current game
    # [1:5]   - cards in player's hand, 0 padded to right
    # [6:25]   - cards played [5 rounds, 4 cards per round]
    # [26:45]    - order played [5 rounds, 4 cards per round]

    def __init__(self):
        self.trump      = 0
        self.mycards    = np.zeros(5)
        self.played     = np.zeros((5, 4))
        self.play_order = np.zeros((5, 4))
    
    def print(self, labels=False):
        pretty_cards = []
        for i in range(5):
#            if self.mycards[i] != 0:
              f = face(self.mycards[i])
              pretty_cards.append(f)
        print("CURRENT SITUATION:")
        print("  Trump:   ", self.trump)  
        print("  My Hand: ", pretty_cards)
        print("  Played:  ")
        print("  Order:   ")

# updates the state with a play
# inputs:
#     round:  the round number of the current game
#     play:   the play number of the current game (0 = leadout)
#     player: the player playing this card
#     card:   the card being played
#     round*5 +1 gets to the head of the array for recording each round and stores the card played
#     round*5 +20 gets to the head of the array for recording the order of play within the round, and stores the player number
    def update(round, play, player, card):
        self.played[round, play] = card 
        self.play_order[round, play] = player
        return None

################################################################################
# CLASS Agent:
#    
#  quick and dirty "strategy" for robotic player imporovement...
#    simply checks to see if:
#       A.  If we can win by moving to a square
#       B.  If we can't win do we need to block the other guy...
#       returns a list of suggested moves

############## NN Model Class ######################
class Agent():
    def __init__(self, state_size, action_size, load=False, filename=''):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = Replay_buffer(state_size, action_size, size=500)
        self.learning_rate = 0.05 # default learning rate   
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.best_move_smoothing = 0.1 # when several next best moves are roughly equal, choose randomly
        self.randocount = 0
        self.NNcount = 0
        self.epsiloncount = 0
        self.movecount = 0
        self.bummers = []
        if load == True:
          self.model = load_model(filename)
#model design:
#       input layer: size of state (27)
#       hidden layers
#       output layer: size of action list (5)        

    def create(self, state_dim, action_dim, hidden_layers=13, hidden_dim=(64, 64, 64), debug=False):
        i = Input(shape=(state_dim,))
        x = i
        for k in range(hidden_layers):
            x = Dense(hidden_dim[k], activation='relu')(x)
        x = Dense(action_dim)(x) # output layer
        self.model = Model(i, x) 
        self.model.compile(optimizer='adam',
                           loss='mse',
                           metrics=['accuracy'])
        if debug:
            print((self.model.summary()))
        return self.model
    
    def model_save(self, filename):
      self.model.save(filename)

    def pred(self, state):
        return self.model.predict(state)

    def act_learner(self, state, player): # state is the current board
        self.movecount += 1
        if np.random.rand() <= self.epsilon:
            self.epsiloncount += 1
            return robot_player_learner(state, player)
        sf = np.zeros((1,9))
        sf[0] = state.board.flatten()

        act = self.model.predict(sf) 
 
        m = get_best_legal_move(state, act[0], self.best_move_smoothing, debug=False)
        self.NNcount += 1
        return m            
    
    def act_trainer(self, state, player): # state is the current board
        self.movecount += 1
        if np.random.rand() <= self.epsilon:
            self.epsiloncount += 1
            return robot_player_teacher(state, player)
        sf = np.zeros((1,9))
        sf[0] = state.board.flatten()

        act = self.model.predict(sf) 
 
        m = get_best_legal_move_with_checking(state, act[0], self.best_move_smoothing, debug=False)
        self.NNcount += 1
        return m            
    
    def update_replay_memory(self, state, action, reward, next_state, done):
        self.memory.store(state, action, reward, next_state, done)
        
    def printmini2(self, mini, t1, t2, t3, t4):
        print("\n")
        print("         current states      a    r              next state        d   tgt             predict(states) pre train                                       Target_full                                                      predict(states) post train                      ")
        print("---------------------------- - ------ ---------------------------- - ------ -------------------------------------------------------------- -------------------------------------------------------------- --------------------------------------------------------------")
        for i in range(len(mini["s"])):
            print("minibatch print def in here")
        print("-----------------------------------\n")
            

    def replay_one_game(self, batch_size=32, debug=False):
        #sample a batch of data from the replay memory
        if self.memory.size < batch_size:
            return
        minibatch = self.memory.sample_batch(batch_size)
        states = minibatch['s']
        actions = minibatch['a']
        rewards = minibatch['r']
        next_states = minibatch['s2']
        done = minibatch['d']
# chanaging the rewards application to happen at end of each game - in the play one game function
        target = rewards # + (1 - done) * self.gamma * future_rwds
        target[done] = rewards[done] # if there are any "done" then replace the target with the reward

        target_full = self.model.predict(states)
        predict_pre = self.model.predict(states)
        target_full[np.arange(len(actions)), actions] = target
        
        self.model.train_on_batch(states, target_full)

        if debug == True:
         predict_post = self.model.predict(states)
         self.printmini2(minibatch, target, predict_pre, target_full, predict_post)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        else:
            self.epsilon = self.epsilon_min
            

class Replay_buffer:
  def __init__(self, obs_dim, act_dim, size):
    self.current_state_buf = np.zeros([size, obs_dim], dtype = np.float32)  # board state pre move
    self.next_state_buf = np.zeros([size, obs_dim], dtype = np.float32)  # board state post move
    self.action_buf = np.zeros(size, dtype = np.uint8)  # board state pre move
    self.reward_buf = np.zeros(size, dtype = np.float32)  # rewards
    self.done_buf = np.zeros(size, dtype = np.uint8)  # board state pre move
    self.ptr, self.size, self.max_size = 0, 0, size

  def store(self, current_state, action, reward, next_state, done):
    self.current_state_buf[self.ptr] = current_state
    self.next_state_buf[self.ptr] = next_state
    self.action_buf[self.ptr] = action
    self.reward_buf[self.ptr] = reward
    self.done_buf[self.ptr] = done
    self.ptr = (self.ptr+1) % self.max_size
    self.size = min(self.size+1, self.max_size)
#    print("Store: ptr=",self.ptr, " size:",self.size)
      
  def sample_batch(self, batch_size=32):  # need to make sure we don't split an EPISODE tho unless we distribute rewards
    idxs = np.random.randint(0, self.size, size=batch_size)
    return dict(s=self.current_state_buf[idxs],
                a=self.action_buf[idxs],
                r=self.reward_buf[idxs],
                s2=self.next_state_buf[idxs],
                d=self.done_buf[idxs])

"""
Miscellaneous helper functions and definitions
"""

#############################################################
## Deck
##
#############################################################
deck_value = {"AH": 5, "KH": 4, "QH": 3, "JH": 2, "TH": 1, \
                   "AD": 5, "KD": 4, "QD": 3, "JD": 2, "TD": 1, \
                   "AS": 5, "KS": 4, "QS": 3, "JS": 2, "TS": 1, \
                   "AC": 5, "KC": 4, "QC": 3, "JC": 2, "TC": 1, \
                   "--": 0}

deck_decode = ["AH", "KH", "QH", "JH", "TH", \
               "AD", "KD", "QD", "JD", "TD", \
               "AS", "KS", "QS", "JS", "TS", \
               "AC", "KC", "QC", "JC", "TC", \
               "--"]
class Deck():
    def __init__(self, cards_in_deck=DECK_SIZE):
        self.cards_in_deck = cards_in_deck

    def create(self):
        self.deck = np.arange(self.cards_in_deck, dtype=int )
        return self.deck

    def deal(self, num_hands=4, cards_per_hand=5, number_of_shuffles=1):
      hands = np.zeros((num_hands,cards_per_hand), dtype=int)
      idx = list(range(0, self.cards_in_deck))
      for i in range(number_of_shuffles):
        random.shuffle(idx)
      for i in range(cards_per_hand):
        for j in range(num_hands):
          hands[j][i] = idx[i*4+j]
      return hands

def suit(card):
    return deck_decode[card][-1]

def face(card):
    return deck_decode[card]

def is_trump(card, trump):
    return suit(card) == trump

def value(card):
    return deck_value[deck_decode[card]]

def pick_trump():
  suits = ['H', 'D', 'C', 'S']
  trump = suits[random.randint(0,3)]
  return trump

def show_hand(hand):
  h = []
  for card in hand:
    h.append(face(card))
  return h

def hands_print(hands, trump, dealer):
  print("Play0    Play1    Play2    Play3\n======   ======   ======   ======")
  for j in range(5):
    print("%3s: %1d   %3s: %1d   %3s: %1d   %3s: %1d" % (face(hands[0][j]), value(hands[0][j]), \
                                                         face(hands[1][j]), value(hands[1][j]), \
                                                         face(hands[2][j]), value(hands[2][j]), \
                                                         face(hands[3][j]), value(hands[3][j])))
  print("Trump: ", trump)
  print("Dealer: ", dealer)


def card_compare(card1, card2, trump):
    suit1 = suit(card1)
    suit2 = suit(card2)
    val1 = value(card1)
    val2 = value(card2)
    
    if is_trump(card1, trump) and (not is_trump(card2, trump)): # card1 was trump, card2 not card 1wins
      return card1
    if is_trump(card2, trump) and (not is_trump(card1, trump)): # card2 was trump, card1 not, so card2 wins  
      return card2
    if is_trump(card1, trump) and is_trump(card2, trump): # both trump, higher value wins
      if val1 >= val2:
        return card1
      else:
        return card2
    if suit1 != suit2:  # no trump - id card2 follow suit?
      return card1        # no, card1 wins
    else:                 # followed suit, so... compare value
      if val2 > val1:     # biggest card wins... card1 unless card2 is bigger
        return card2
      else:
         return card1
    print("ERROR IN card_compare: ", card1, card2)
    return -1          

def who_won_round(trump, round_state, play_order, debug=0):
  # evaluate cards played to determine winner
  # if trump played, highest trump wins
  # if no_trump played, highest of lead suit played wins
  best_card = round_state[0]
  best_player = play_order[0]
  for i in range(len(round_state)):
    if debug > 2:
      print("player ", play_order[i], " played ", face(round_state[i]))
    test_card = card_compare(best_card, round_state[i], trump)
    if test_card != best_card:
      best_card = test_card
      best_player = play_order[i]
  return best_player, best_card

def what_is_winning_round(trump, round_state, debug=0):
  # evaluate cards played to determine current winning card
  # if trump played, highest trump wins
  # if no_trump played, highest of lead suit played is winning

  best_card = round_state[0]
  if len(round_state) == 1:
    return best_card
  if debug > 3: 
    print("what is winning: ", round_state, len(round_state))
  for i in range(1, len(round_state)):
    test_card = card_compare(best_card, round_state[i], trump)
    if test_card != best_card:
      best_card = test_card
  return best_card

"""
MAIN ACTION SECTION:
  - play_one_card
  - play_one_hand
  - play_one_game
"""
def play_one_card_bot(hands, trump, player, round_state, debug=0):
  # if am i lead?
  #   play winner or slough? 
  # else can i follow suit?
  # else do I slough... or trump!
  
  # preprocess my hand
    mycards = np.where(hands[player] != CARD_PLAYED)
    playable = hands[player][mycards]
    card_face = []
    card_value = []
    card_suit = []
    card_trump = []
    card_index = []
    
    for card in playable:
      card_face.append(face(card))
      card_value.append(value(card))
      card_suit.append(suit(card))    
      card_trump.append(is_trump(card, trump))
      card_index.append(card)

    if len(round_state) == 0:  # i'm lead...
      if debug > 3: 
        print("I'm lead, my cards: ", show_hand(playable))
      # play highest non-trump
      idx = [i for i in range(len(card_value)) ]
      high = 0
      high_idx = 0
      card_idx = 0
      if len(idx) > 0: # we can follow suit
        if debug > 3:
          print("Following suit...", idx)
        for i in idx:
          if high < card_value[i]:
              high = card_value[i]
              high_idx = i    
      play = card_index[high_idx]
      hands[player][mycards[0][high_idx]] = CARD_PLAYED
      suit_to_follow = suit(play)

    else:
      suit_to_follow = suit(round_state[0])
      if debug > 3:
        print("I'm follower, trump is: ", trump, "suit to follow is: ", suit_to_follow, "my cards: ", show_hand(playable))
      # play highest non-trump in suit
      # check to see winning card in current play
      current_winning_card = what_is_winning_round(trump, round_state)
      if debug > 3:
        print("Current winning card: ", face(current_winning_card), is_trump(current_winning_card, trump))
      #if current winning card is trump:
      # if we have to follow suit, go low
      # if we have to slough go short suit
      # if we can over-trump, do it
      follow_go_low = False
      over_trump = False

      if is_trump(current_winning_card, trump):
        follow_go_low = True
        over_trump = True

      idx = [i for i in range(len(playable)) if card_suit[i] == suit_to_follow ]
      high = 0
      low = 100
      card_idx = 0
      if len(idx) > 0: # we can follow suit
        if debug > 3:
          print("Following suit...", idx)
        for i in idx:
          if high < card_value[i]:
              high = card_value[i]
              high_idx = i    
          if low > card_value[i]:
              low = card_value[i]
              low_idx = i
        # we have our highest and lowest suit followers
        if debug > 3:
          print("high/low follwer: ", face(card_index[high_idx]), face(card_index[low_idx]))

        if value(current_winning_card) > value(card_index[high_idx]): # we can't beat the current winning card, so go low in following suit
          follow_go_low = True

        if follow_go_low:
          card_idx = low_idx
          if debug > 4:
            print("going low")
        else:
          card_idx = high_idx
          if debug > 4:
            print("going high")

      else: # can't follow suit... can we trump it?
        idx = [i for i in range(len(card_trump)) if card_trump[i] == True]
        if len(idx) > 0: # we can trump
          if len(idx) == 1: # we only have one trump, so use it
            card_idx = idx[0]
          else:  # we have more than one trump - choose

            high = 0
            low = 100
            card_idx = 0
            for i in idx: # find my high and low trump
              if high < card_value[i]:
                high = card_value[i]
                high_idx = i
              if low > card_value[i]:
                low = card_value[i]
                low_idx = i  
            if over_trump:
                card_idx = high_idx
                if debug > 4:
                  print("Over Trump it!", card_idx)
            else: # play low trump
                 card_idx = low_idx
                 if debug > 4:
                   print("Under Trump it!", card_idx)
        else:  # no trump, let's slough a loser
          idx = [i for i in range(len(playable))] # get list of cards
          if len(idx) > 0: # we can slough
              low = 100
              card_idx = 0
              for i in idx: # play lowest slough
                if low > card_value[i]:
                   low = card_value[i]
                   card_idx = i

      play = card_index[card_idx]
      if debug > 2:
        print("Playing: ", face(card_index[card_idx]))
      hands[player][mycards[0][card_idx]] = CARD_PLAYED
    return play, hands



def play_one_card(hands, trump, player, round_state, debug=0):
  # if am i lead?
  #   play winner or slough? 
  # else can i follow suit?
  # else do I slough... or trump!
  
  # preprocess my hand
    mycards = np.where(hands[player] != CARD_PLAYED)
    playable = hands[player][mycards]
    card_face = []
    card_value = []
    card_suit = []
    card_trump = []
    card_index = []
    
    for card in playable:
      card_face.append(face(card))
      card_value.append(value(card))
      card_suit.append(suit(card))    
      card_trump.append(is_trump(card, trump))
      card_index.append(card)

    if len(round_state) == 0:  # i'm lead...
      if debug > 4: 
        print("I'm lead, my cards: ", show_hand(playable))
      # play highest non-trump
      idx = [i for i in range(len(card_value)) ]
      high = 0
      high_idx = 0
      card_idx = 0
      if len(idx) > 0: # we can follow suit
        if debug > 4:
          print("Following suit...", idx)
        for i in idx:
          if high < card_value[i]:
              high = card_value[i]
              high_idx = i    
      play = card_index[high_idx]
      hands[player][mycards[0][high_idx]] = CARD_PLAYED
      suit_to_follow = suit(play)

    else:
      suit_to_follow = suit(round_state[0])
      if debug > 4:
        print("I'm follower, trump is: ", trump, "suit to follow is: ", suit_to_follow, "my cards: ", show_hand(playable))
      # play highest non-trump in suit
      # check to see winning card in current play
      current_winning_card = what_is_winning_round(trump, round_state)
      if debug > 4:
        print("Current winning card: ", face(current_winning_card), is_trump(current_winning_card, trump))
      #if current winning card is trump:
      # if we have to follow suit, go low
      # if we have to slough go short suit
      # if we can over-trump, do it
      follow_go_low = False
      over_trump = False

      if is_trump(current_winning_card, trump):
        follow_go_low = True
        over_trump = True

      idx = [i for i in range(len(playable)) if card_suit[i] == suit_to_follow ]
      high = 0
      low = 100
      card_idx = 0
      if len(idx) > 0: # we can follow suit
        if debug > 4:
          print("Following suit...", idx)
        for i in idx:
          if high < card_value[i]:
              high = card_value[i]
              high_idx = i    
          if low > card_value[i]:
              low = card_value[i]
              low_idx = i
        # we have our highest and lowest suit followers
        if debug > 4:
          print("high/low follwer: ", face(card_index[high_idx]), face(card_index[low_idx]))

        if value(current_winning_card) > value(card_index[high_idx]): # we can't beat the current winning card, so go low in following suit
          follow_go_low = True

        if follow_go_low:
          card_idx = low_idx
          if debug > 4:
            print("going low")
        else:
          card_idx = high_idx
          if debug > 4:
            print("going high")

      else: # can't follow suit... can we trump it?
        idx = [i for i in range(len(card_trump)) if card_trump[i] == True]
        if len(idx) > 0: # we can trump
          if len(idx) == 1: # we only have one trump, so use it
            card_idx = idx[0]
          else:  # we have more than one trump - choose

            high = 0
            low = 100
            card_idx = 0
            for i in idx: # find my high and low trump
              if high < card_value[i]:
                high = card_value[i]
                high_idx = i
              if low > card_value[i]:
                low = card_value[i]
                low_idx = i  
            if over_trump:
                card_idx = high_idx
                if debug > 4:
                  print("Over Trump it!", card_idx)
            else: # play low trump
                 card_idx = low_idx
                 if debug > 4:
                   print("Under Trump it!", card_idx)
        else:  # no trump, let's slough a loser
          idx = [i for i in range(len(playable))] # get list of cards
          if len(idx) > 0: # we can slough
              low = 100
              card_idx = 0
              for i in idx: # play lowest slough
                if low > card_value[i]:
                   low = card_value[i]
                   card_idx = i

      play = card_index[card_idx]
      if debug > 4:
        print("Playing: ", face(card_index[card_idx]))
      hands[player][mycards[0][card_idx]] = CARD_PLAYED
    return play, hands

def play_one_game(deck, dealer, num_players=4, debug=0):
  # dealer = passed in from overall main game
  # instantiate State for each player
  # deal cards
  # pick trump
  # lead = (dealer + 1) % 4
  # for each round (5), for each player (4) play_one_card
  # rotate the dealer for the next round

  hands = deck.deal(num_hands=4, cards_per_hand=5, number_of_shuffles=2)
  trump = pick_trump()
  playerstate = [0] * num_players

  for i in range(num_players):
    playerstate[i] = State() # initialize state object for each player
    playerstate[i].mycards = hands[i]
    playerstate[i].trump = trump

  if debug > 4:
    hands_print(hands, trump, dealer)
  if debug > 1:  
    for i in range(num_players):
      playerstate[i].print()

  tricks = [0] * num_players
  lead = (dealer + 1) % num_players # initial lead to dealers left
  
  for round in range(5):
    round_state = []
    round_order = []
    for p in range(lead, lead+num_players):
      player = p % num_players
      if debug > 4:
        print("ROUND: ", round, "Player: ", player)
      play, hands = play_one_card(hands, trump, player, round_state)

      if debug > 4:
        print("Player %1d plays %3s" % (player, face(play)))
      round_state.append(play)
      round_order.append(player)
      faces = []
      for i in range(len(round_state)):
        faces.append(face(round_state[i]))
      if debug > 4:
          print("Played so far: ", faces)
          print("Round Order:   ", round_order)
          print("-------------------")
      winner, c = who_won_round(trump, round_state, round_order)  
    if debug > 4:
      print("Trump was: ", trump)
      print("winning card was: ", face(c), "played by: ", winner)
      print("====================================\n")
    tricks[winner] += 1
    lead = winner
  if debug > 2:
    print("----- End of Game Results -----")
    for i in range(num_players):
      print("Player ", i, "took ", tricks[i], "tricks.")
    print("-------------------------------")

  return tricks, round_state



In [61]:
def play_one_tournament(winning_score=10, num_players=4, debug=0):
  deck = Deck(DECK_SIZE)
  score = [0] * num_players
  dealer = np.random.randint(0,num_players)  # start each tournament with random dealer
  games_played_this_tournament = 0

  while True:
    if (max(score) >= winning_score): 
      # ensure no ties
      w = max(score)
      if score.count(w) == 1:
        break

    deck.create()  
    tricks, played = play_one_game(deck, dealer, num_players, debug)
    for i in range(num_players):
      score[i] += tricks[i]  
    dealer = (dealer + 1) % num_players
    games_played_this_tournament += 1

  if debug > 0:
        print("\n========= End of Tournament Score ========Games: ", games_played_this_tournament)
        for i in range(num_players):
          print("Player ", i, "Score ", score[i])
        print("\n===========================================")

  return score, games_played_this_tournament

In [66]:
###############################
# Testing loop
################################
#game hyperparameters
num_players = 4
win_score = 15
debug_level = 2

#stats initialization
scores = [0] * num_players    
tournaments = 1000
tournaments_won = [0] * num_players
tricks_in_tournament = 0
total_games_played = 0

for i in range(tournaments):    
    score, games_played_this_tournament = play_one_tournament(win_score, num_players, debug=debug_level)
    for j in range(num_players):
      scores[j] += score[j]
    tricks_in_tournament += sum(score)
    tournament_winner = np.argmax(score)
    tournaments_won[tournament_winner] += 1
    total_games_played += games_played_this_tournament

    if ((i + 1) % 1000) == 0:
      print("\n========= OVERALL Tournament wins %4d/%4d ==========" % (i+1, tournaments))
      print("Player Wins   Win pct")
      for j in range(num_players):
        print("  %1d    %5d    %2.2f" %  (j+1, tournaments_won[j], tournaments_won[j]/sum(tournaments_won)))
      print("\n========= OVERALL Tricks taken  out of %4d==========" % (sum(scores)))
      print("Player Hands  Win pct")
      for j in range(num_players):
        print("  %1d    %5d    %2.2f" %  (j+1, scores[j], scores[j]/sum(scores)))
      print("\n=========================================")

      print(" Total tricks played: ", tricks_in_tournament)
      print(" Total games played: ", total_games_played)
      print(" Total tournaments played: ", sum(tournaments_won))
      print(" Average tricks per Tournament: ", tricks_in_tournament / sum(tournaments_won))
      print(" Average games per Tournament: ", total_games_played / sum(tournaments_won))
      print(" Average tricks per game: ", tricks_in_tournament / total_games_played)


Streaming output truncated to the last 5000 lines.
  Order:   
CURRENT SITUATION:
  Trump:    C
  My Hand:  ['JD', 'JS', 'QS', 'JH', 'KD']
  Played:  
  Order:   
CURRENT SITUATION:
  Trump:    C
  My Hand:  ['KS', 'QH', 'QD', 'TS', 'TC']
  Played:  
  Order:   
CURRENT SITUATION:
  Trump:    C
  My Hand:  ['AH', 'TD', 'TH', 'JC', 'QC']
  Played:  
  Order:   
CURRENT SITUATION:
  Trump:    C
  My Hand:  ['KC', 'AC', 'AD', 'KH', 'AS']
  Played:  
  Order:   

========= End of Tournament Score ========Games:  11
Player  0 Score  13
Player  1 Score  13
Player  2 Score  14
Player  3 Score  15

CURRENT SITUATION:
  Trump:    S
  My Hand:  ['JH', 'AH', 'JS', 'TC', 'QD']
  Played:  
  Order:   
CURRENT SITUATION:
  Trump:    S
  My Hand:  ['KH', 'KS', 'AD', 'QC', 'TD']
  Played:  
  Order:   
CURRENT SITUATION:
  Trump:    S
  My Hand:  ['AS', 'AC', 'TS', 'JD', 'JC']
  Played:  
  Order:   
CURRENT SITUATION:
  Trump:    S
  My Hand:  ['TH', 'KD', 'QS', 'KC', 'QH']
  Played:  
  Order:   
CU